In [ ]:
# default_exp recentFeed

# recentFeed

> Parse the recent filings feed.

In [ ]:
#hide
%load_ext autoreload
%autoreload 2
from nbdev import show_doc

In [ ]:
#export

import re
import xml.etree.cElementTree as cElTree

from secscan import utils

In [ ]:
# uncomment and run to regenerate all library Python files
# from nbdev.export import notebook2script; notebook2script()